## This notebook shows the fastest mean to select some spots with a simple filter
* It is possible to interact with external libraries, have a look on the `init_diagram` notebook.
* To instanciate a `Diagram` allows you to apply efficiently filters based on image processing.
* They are several means to instanciate a `diagram` some of them are betters than overs.

In [ ]:
%matplotlib notebook

import multiprocessing.pool
import pathlib

import matplotlib.pyplot as plt
import torch
import tqdm

# sys.path.insert(0, "/data/bm32/inhouse/laueimproc")
# sys.path.insert(0, "/data/bm32/inhouse/laueimprocenv/lib/python3.11/site-packages")
from laueimproc.io.download import get_samples  # gives access to the dataset
from laueimproc import Diagram

## Set and take control of the global configuration parameters
* The cache is automaticaly managed, but you can control it a bit.

In [ ]:
# manage context

from laueimproc.opti.manager import DiagramManager

manager = DiagramManager()
manager.verbose = True  # to display the operations in background
manager.ram_limit = 0.8  # trigger cache clear after 80% of ram full, default based on swapiness
manager.update()  # dereference unreachable diagrams

## Initialise the diagrams
* The different ways of initializing a diagram are described in detail in the `init_diagram` notebook.

In [ ]:
all_files = list(get_samples().iterdir())  # the list of all images path
# all_files = list(pathlib.Path("my_folder/").iterdir())
diagrams = [Diagram(f) for f in all_files]
for diagram in diagrams:
    diagram.find_spots(density=0.55)
    # diagram.plot(plt.figure(layout="tight", figsize=(10, 10))); plt.show()

## Fit spots roi with gaussians

In [ ]:
for diagram in tqdm.tqdm(diagrams):
    # intensities = diagram.compute_pxl_intensities()
    # diagram.filter_spots(intensities > intensities.median())
    
    mean, cov, infodict = diagram.fit_gaussian()
    # print(infodict["mse"].shape)
    
    # indexs = torch.argsort(more["aic"], descending=False)[:10]
    # diagram.filter_spots(indexs)
    # diagram.plot(plt.figure(layout="tight", figsize=(10, 10))); plt.show()

## Keeps the 10 most intense spots for each diagram